In [1]:

# LangGraph官方文档教程 
# https://langchain-ai.github.io/langgraph/tutorials/get-started/5-customize-state/


In [2]:
# Customize state¶
# 自定义状态 ¶
# In this tutorial, you will add additional fields to the state to define complex behavior without relying on the message list. The chatbot will use its search tool to find specific information and forward them to a human for review.
# 在本教程中，您将向状态添加其他字段，以定义复杂的行为，而无需依赖消息列表。聊天机器人将使用其搜索工具查找特定信息并将其转发给人工审核。

In [3]:
# Note  笔记

# This tutorial builds on Add human-in-the-loop controls.
# 本教程以添加人机交互控制为基础。

In [4]:
import getpass
import os
from langchain.chat_models import init_chat_model

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

llm = init_chat_model("google_genai:gemini-2.0-flash")

In [5]:
# 1. Add keys to the state¶
# 1. 向状态添加键 ¶
# Update the chatbot to research the birthday of an entity by adding name and birthday keys to the state:
# 通过向状态添加 name 和 birthday 键来更新聊天机器人以研究实体的生日：

# API Reference: add_messages
# API 参考：add_messages


In [6]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt


class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

In [7]:
# Adding this information to the state makes it easily accessible by other graph nodes (like a downstream node that stores or processes the information), as well as the graph's persistence layer.
# 将此信息添加到状态中，使得其他图形节点（如存储或处理信息的下游节点）以及图形的持久层可以轻松访问它。


In [8]:
# 2. Update the state inside the tool¶
# 2. 更新工具内部的状态 ¶
# Now, populate the state keys inside of the human_assistance tool. This allows a human to review the information before it is stored in the state. Use Command to issue a state update from inside the tool.
# 现在，在 human_assistance 工具中填充状态键。这允许人类在信息存储到状态之前对其进行审核。使用 Command 从工具内部发出状态更新。


In [9]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)

In [10]:
# The rest of the graph stays the same.
# 图表的其余部分保持不变。


In [11]:
tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [12]:
# 3. Prompt the chatbot¶
# 3. 提示聊天机器人 ¶
# Prompt the chatbot to look up the "birthday" of the LangGraph library and direct the chatbot to reach out to the human_assistance tool once it has the required information. By setting name and birthday in the arguments for the tool, you force the chatbot to generate proposals for these fields.
# 提示聊天机器人查找 LangGraph 库的“生日”，并在获得所需信息后指示聊天机器人联系 human_assistance 工具。通过在该工具的参数中设置 name 和 birthday ，您可以强制聊天机器人为这些字段生成提案。



In [13]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (1c6ed0f6-1bb7-4452-a5ed-8cd17667b698)
 Call ID: 1c6ed0f6-1bb7-4452-a5ed-8cd17667b698
  Args:
    query: LangGraph release date
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph release date", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://pypi.org/project/langgraph/", "title": "langgraph·PyPI", "content": "langgraph 0.4.8 Image 5: LangGraph Logo Install LangGraph: from langgraph.prebuilt import create_react_agent Or, to learn how to build an agent workflow with a customizable architecture, long-term memory, and other complex task handling, see the LangGraph basics tutorials.

In [14]:
# We've hit the interrupt in the human_assistance tool again.
# 我们再次遇到了 human_assistance 工具中的 interrupt 。


In [15]:
# 4. Add human assistance¶
# 4. 添加人工协助 ¶
# The chatbot failed to identify the correct date, so supply it with information:
# 聊天机器人无法识别正确的日期，因此请向其提供以下信息：


In [16]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

Based on the search results, I found information about LangGraph versions and releases on PyPI and GitHub. However, I couldn't find a specific initial release date. I will use the human_assistance tool to get help.
Tool Calls:
  human_assistance (ab93b97d-dcbd-4c54-8b1a-d95e5369cdab)
 Call ID: ab93b97d-dcbd-4c54-8b1a-d95e5369cdab
  Args:
    name: LangGraph Release Date Inquiry
    birthday: Unknown
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================

LangGraph was released on January 17, 2024.


In [17]:
# Note that these fields are now reflected in the state:
# 请注意，这些字段现在反映在状态中：

In [18]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

In [19]:
# This makes them easily accessible to downstream nodes (e.g., a node that further processes or stores the information).
# 这使得下游节点（例如，进一步处理或存储信息的节点）可以轻松访问它们。



In [20]:
# 5. Manually update the state¶
# 5. 手动更新状态 ¶
# LangGraph gives a high degree of control over the application state. For instance, at any point (including when interrupted), you can manually override a key using graph.update_state:
# LangGraph 对应用程序状态提供了高度的控制。例如，在任何时候（包括中断时），你都可以使用 graph.update_state 手动覆盖某个键：


In [21]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f060e8c-5cf3-646f-8006-88ef774909cb'}}

In [22]:
# 6. View the new value¶
# 6. 查看新值 ¶
# If you call graph.get_state, you can see the new value is reflected:
# 如果你调用 graph.get_state ，你可以看到新的值被反映出来：


In [23]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}

In [24]:
# Manual state updates will generate a trace in LangSmith. If desired, they can also be used to control human-in-the-loop workflows. Use of the interrupt function is generally recommended instead, as it allows data to be transmitted in a human-in-the-loop interaction independently of state updates.
# 手动状态更新会在 LangSmith 中生成跟踪 。如果需要，它们也可以用来控制人机交互的工作流程 。通常建议使用 interrupt 函数，因为它允许在人机交互中传输数据，而不受状态更新的影响。

# Congratulations! You've added custom keys to the state to facilitate a more complex workflow, and learned how to generate state updates from inside tools.
# 恭喜！ 您已向状态添加了自定义键以方便更复杂的工作流程，并了解了如何从工具内部生成状态更新。

